In [ ]:
#该代码需要已分类出高压、低压新装、增容、销户、减容数据，以及电量数据的excel文件，单位为万千伏安、万千瓦时
#该代码块对电量数据进行季节性调整后一阶差分、对业扩数据进行一阶差分
#输出文件为经过CCF分析的结果，得到滞后期数据，影响系数数据

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL # 导入STL
from statsmodels.tsa.stattools import ccf
from IPython.display import display
import re

# ==============================================================================
# 0. 参数配置
# ==============================================================================
# 指向您的【原始】数据文件
FILE_PATH = r'E:\A智网\业扩分析\8月分析\7月业扩月度报告\8.15\7月业扩月底电量报告全行业_净值.xlsx' 
# 新的输出文件名，以反映处理方法
OUTPUT_PATH = r'E:\A智网\业扩分析\8月分析\7月业扩月度报告\8.15\7月业扩月底电量报告全行业_净值_业扩分析结果.xlsx'
SHEET_MAP = {
    'kwh': '电量',
    'cap_high_new': '高压净新装',
    'cap_high_exp': '高压净增容',
    'cap_low_new': '低压净新装',
    'cap_low_exp': '低压净增容'
}

# ==============================================================================
# 1. 数据加载与预处理函数 (封装复杂性)
# ==============================================================================
def load_and_prepare_data(file_path, sheet_map):
    """
    加载所有原始数据Sheet，进行清洗和格式转换，返回一个包含所有数据的字典。
    """
    print("--- 步骤 1: 正在加载并预处理所有原始数据 ---")
    all_data = {}
    for var_name, sheet_name in sheet_map.items():
        try:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            df['分类'] = df['分类'].astype(str).str.strip()
            valid_date_cols = [col for col in df.columns if re.match(r'^\d{6}$', str(col).split('.')[0])]
            df = df[['分类'] + valid_date_cols]
            df_long = df.melt(id_vars='分类', var_name='date', value_name=var_name)
            df_long['date'] = pd.to_datetime(df_long['date'].astype(str), format='%Y%m')
            df_long[var_name] = pd.to_numeric(df_long[var_name], errors='coerce').fillna(0)
            all_data[var_name] = df_long.set_index(['分类', 'date'])
            print(f"  -> Sheet '{sheet_name}' 加载并转换成功。")
        except Exception as e:
            raise RuntimeError(f"处理Sheet '{sheet_name}' 时出错: {e}")
    
    df_full = pd.concat(all_data.values(), axis=1)
    return df_full

# ==============================================================================
# 2. 主分析流程
# ==============================================================================
def run_full_analysis(df_full):
    """
    对加载好的完整数据，分行业进行 预处理 -> CCF -> 回归建模 的完整流程。
    """
    print("\n--- 步骤 2: 开始分行业进行完整分析流程 ---")
    
    industry_list = df_full.index.get_level_values('分类').unique().tolist()
    print(f"将对以下 {len(industry_list)} 个行业进行分析。")
    
    final_results = []

    for industry in industry_list:
        print(f"\n{'='*25} 正在分析: {industry} {'='*25}")
        
        df_industry_raw = df_full.loc[industry]
        
        # --- B. 序列平稳化 (STL + 差分版) ---
        df_processed = pd.DataFrame()
        
        # ★★★★★★★★★★★ 关键修改点 ★★★★★★★★★★★
        # 对电量数据进行STL季节性调整后，再进行一阶差分
        kwh_ts = df_industry_raw['kwh']
        try:
            # 检查数据是否适合进行STL分解
            if len(kwh_ts.dropna()) < 25 or kwh_ts.std() < 1e-6:
                print("    -> 电量数据不足或无波动，仅进行一阶差分。")
                df_processed['d_kwh'] = kwh_ts.diff(1)
            else:
                print("    -> 正在对电量数据进行STL季节性调整...")
                stl = STL(kwh_ts, period=12, robust=True)
                res = stl.fit()
                kwh_seasonally_adjusted = res.trend + res.resid
                df_processed['d_kwh'] = kwh_seasonally_adjusted.diff(1)
                print("    -> STL调整并一阶差分完成。")
                
        except Exception as e:
            print(f"    -> 警告: STL处理失败 ({e})，回退到仅进行一阶差分。")
            df_processed['d_kwh'] = kwh_ts.diff(1)
        # ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

        # 对业扩数据进行一阶差分
        for cap_var in [c for c in df_industry_raw.columns if c != 'kwh']:
            df_processed[cap_var] = df_industry_raw[cap_var].diff(1)
        
        df_processed.dropna(inplace=True)
        
        if len(df_processed) < 30:
            print("    -> 预处理后数据点不足 (< 30)，跳过。")
            continue
        print(f"    -> 数据平稳化完成，剩余 {len(df_processed)} 个有效数据点。")
        
        # --- C. CCF分析确定滞后期 ---
        lags = {}
        exog_vars = [c for c in df_processed.columns if c != 'd_kwh']
        for var in exog_vars:
            ccf_vals = ccf(df_processed['d_kwh'], df_processed[var], adjusted=False)
            relevant_lags = range(1, 9)
            best_lag = relevant_lags[np.argmax(np.abs(ccf_vals[1:9]))]
            lags[var] = best_lag
        print(f"    -> CCF分析完成，最佳滞后期: {lags}")

        # --- D. 回归建模 ---
        df_model = df_processed[['d_kwh']].copy()
        for var, lag in lags.items():
            df_model[f'{var}_lag{lag}'] = df_processed[var].shift(lag)
        df_model.dropna(inplace=True)
        
        if len(df_model) < 25:
            print("    -> 创建滞后变量后数据点不足 (< 25)，跳过。")
            continue
            
        Y = df_model['d_kwh']
        X = df_model.drop(columns='d_kwh')
        X = sm.add_constant(X)
        model_results = sm.OLS(Y, X).fit()
        
        # --- E. 提取结果 ---
        result_row = {'行业分类': industry, 'R-squared_adj': model_results.rsquared_adj}
        for var, lag in lags.items():
            result_row[f'k_{var}'] = lag
            param_name = f'{var}_lag{lag}'
            result_row[f'β_{var}'] = model_results.params.get(param_name, np.nan)
            result_row[f'p_{var}'] = model_results.pvalues.get(param_name, np.nan)
        
        final_results.append(result_row)
        print(f"    -> 回归建模完成，调整后R²: {model_results.rsquared_adj:.3f}")

    return pd.DataFrame(final_results)

# ==============================================================================
# 3. 结果展示与保存
# ==============================================================================
def display_and_save_results(df_summary, output_path):
    if df_summary.empty:
        print("\n未能完成任何行业的分析。")
        return

    print(f"\n{'='*30} 所有行业分析结果汇总 {'='*30}")
    df_summary.set_index('行业分类', inplace=True)

    lag_cols = sorted([c for c in df_summary.columns if c.startswith('k_')])
    df_lags = df_summary[lag_cols].rename(columns=lambda c: c.replace('k_', ''))
    print("\n各行业【最佳影响滞后期(月)】汇总:")
    display(df_lags)
    
    beta_cols = sorted([c for c in df_summary.columns if c.startswith('β_')])
    df_betas = df_summary[beta_cols].rename(columns=lambda c: c.replace('β_', ''))
    print("\n各行业【影响系数(β值 - kWh/kVA)】汇总 (绿色粗体表示统计显著):")
    
    def highlight_significant(x):
        styles = pd.DataFrame('', index=x.index, columns=x.columns)
        for beta_col in x.columns:
            pval_col = 'p_' + beta_col
            if pval_col in df_summary.columns:
                styles[beta_col] = np.where(df_summary[pval_col] < 0.1, 
                                            'font-weight: bold; color: green;', 
                                            'color: grey;')
        return styles

    styled_betas = df_betas.style.format("{:.2f}").apply(highlight_significant, axis=None)
    display(styled_betas)
    
    with pd.ExcelWriter(output_path) as writer:
        df_lags.to_excel(writer, sheet_name='滞后期(k)汇总')
        df_betas.to_excel(writer, sheet_name='影响系数(β)汇总')
        df_summary.to_excel(writer, sheet_name='全部详细结果')
    print(f"\n完整汇总结果已保存到: {output_path}")

# ==============================================================================
# 4. 执行主程序
# ==============================================================================
if __name__ == '__main__':
    # 步骤1: 加载和准备数据
    full_data = load_and_prepare_data(FILE_PATH, SHEET_MAP)
    
    # 步骤2: 运行完整分析流程
    summary_results = run_full_analysis(full_data)
    
    # 步骤3: 展示并保存结果
    display_and_save_results(summary_results, OUTPUT_PATH)

    print("\n--- 全部分析任务完成 ---")

--- 步骤 1: 正在加载并预处理所有原始数据 ---
  -> Sheet '电量' 加载并转换成功。
  -> Sheet '高压净新装' 加载并转换成功。
  -> Sheet '高压净增容' 加载并转换成功。
  -> Sheet '低压净新装' 加载并转换成功。
  -> Sheet '低压净增容' 加载并转换成功。

--- 步骤 2: 开始分行业进行完整分析流程 ---
将对以下 133 个行业进行分析。

========================= 正在分析: 全行业总计 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 3, 'cap_high_exp': 1, 'cap_low_new': 6, 'cap_low_exp': 5}
    -> 回归建模完成，调整后R²: 0.116

========================= 正在分析: A、全行业用电合计 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 5, 'cap_high_exp': 1, 'cap_low_new': 2, 'cap_low_exp': 5}
    -> 回归建模完成，调整后R²: 0.104

========================= 正在分析: 第一产业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 6, 'cap_high_exp': 7, 'cap_low_new': 7, 'cap_low_exp': 4}
    -> 回归建

d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value encountered in divide
  ret = cvf / (np.std(x) * np.std(y))
d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value encountered in divide
  ret = cvf / (np.std(x) * np.std(y))
d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value encountered in divide
  ret = cvf / (np.std(x) * np.std(y))


    -> 回归建模完成，调整后R²: 0.417

========================= 正在分析: 6.其他采矿业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 1, 'cap_high_exp': 4, 'cap_low_new': 8, 'cap_low_exp': 6}
    -> 回归建模完成，调整后R²: 0.454

========================= 正在分析: （二）制造业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 4, 'cap_high_exp': 1, 'cap_low_new': 8, 'cap_low_exp': 7}
    -> 回归建模完成，调整后R²: 0.238

========================= 正在分析: 1.农副食品加工业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 3, 'cap_high_exp': 6, 'cap_low_new': 2, 'cap_low_exp': 6}
    -> 回归建模完成，调整后R²: 0.198

========================= 正在分析: 2.食品制造业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期

d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value encountered in divide
  ret = cvf / (np.std(x) * np.std(y))
d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value encountered in divide
  ret = cvf / (np.std(x) * np.std(y))
d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value encountered in divide
  ret = cvf / (np.std(x) * np.std(y))
d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value encountered in divide
  ret = cvf / (np.std(x) * np.std(y))
d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value encountered in divide
  ret = cvf / (np.std(x) * np.std(y))
d:\Software\Anaconda\envs\dianli\Lib\site-packages\statsmodels\tsa\stattools.py:1179: RuntimeWarning: invalid value enco

    -> 回归建模完成，调整后R²: 0.291

========================= 正在分析: 13.石油、煤炭及其他燃料加工业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 7, 'cap_high_exp': 4, 'cap_low_new': 1, 'cap_low_exp': 8}
    -> 回归建模完成，调整后R²: 0.036

========================= 正在分析: 其中：煤化工 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 3, 'cap_high_exp': 7, 'cap_low_new': 5, 'cap_low_exp': 1}
    -> 回归建模完成，调整后R²: -0.014

========================= 正在分析: 14.化学原料和化学制品制造业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 3, 'cap_high_exp': 1, 'cap_low_new': 3, 'cap_low_exp': 6}
    -> 回归建模完成，调整后R²: 0.119

========================= 正在分析: 其中：氯碱 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    ->

...

    -> 回归建模完成，调整后R²: 0.232

========================= 正在分析: 八、金融业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 2, 'cap_high_exp': 7, 'cap_low_new': 1, 'cap_low_exp': 6}
    -> 回归建模完成，调整后R²: 0.091

========================= 正在分析: 九、房地产业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 8, 'cap_high_exp': 1, 'cap_low_new': 7, 'cap_low_exp': 5}
    -> 回归建模完成，调整后R²: 0.420

========================= 正在分析: 十、租赁和商务服务业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: {'cap_high_new': 1, 'cap_high_exp': 1, 'cap_low_new': 2, 'cap_low_exp': 1}
    -> 回归建模完成，调整后R²: 0.265

========================= 正在分析: 其中：租赁业 =========================
    -> 正在对电量数据进行STL季节性调整...
    -> STL调整并一阶差分完成。
    -> 数据平稳化完成，剩余 42 个有效数据点。
    -> CCF分析完成，最佳滞后期: 

,cap_high_exp,cap_high_new,cap_low_exp,cap_low_new
行业分类,,,,
全行业总计,1,3,5,6
A、全行业用电合计,1,5,5,2
第一产业,7,6,4,7
第二产业,3,6,7,5
第三产业,8,2,2,3
...,...,...,...,...
3.居民服务、修理和其他服务业,1,1,6,1
4.教育、文化、体育和娱乐业,7,3,2,5
其中：教育,7,3,2,5



各行业【影响系数(β值 - kWh/kVA)】汇总 (绿色粗体表示统计显著):


,cap_high_exp,cap_high_new,cap_low_exp,cap_low_new
行业分类,,,,
全行业总计,0.09,-0.00,-0.27,0.07
A、全行业用电合计,0.14,0.00,-0.06,-0.06
第一产业,-0.21,-0.04,-0.25,0.03
第二产业,-0.13,0.01,-1.28,0.59
第三产业,-0.14,-0.02,0.17,-0.06
B、城乡居民生活用电合计,1.23,0.29,1.67,0.04
城镇居民,-0.45,0.31,-4.01,-0.05
乡村居民,2.48,-0.90,1.46,0.07
全行业用电分类,0.14,0.00,-0.06,-0.06



完整汇总结果已保存到: E:\A智网\业扩分析\8月分析\7月业扩月度报告\8.15\7月业扩月底电量报告全行业_净值_业扩分析结果.xlsx

--- 全部分析任务完成 ---


In [ ]:
#根据上一步CCF得出的数据，进行进一步分析可以预测出未来月份的业扩贡献电量（目前不准确）

import pandas as pd
import numpy as np
import re
from IPython.display import display

# ==============================================================================
# 0. 参数配置
# ==============================================================================
results_file_path = r'E:\A智网\业扩分析\8月分析\7月业扩月度报告\8.15\7月业扩月底电量报告全行业_净值_业扩分析结果.xlsx'
raw_cap_data_path = r'E:\A智网\业扩分析\8月分析\7月业扩月度报告\8.15\7月业扩月底电量报告全行业_净值.xlsx'

prediction_months = ['202507', '202508', '202509']  #此处修改为需要预测的月份
cap_sheet_map = {
    'cap_high_new': '高压净新装',
    'cap_high_exp': '高压净增容',
    'cap_low_new': '低压净新装',
    'cap_low_exp': '低压净增容'
}

# ==============================================================================
# 1. 加载数据 (带Sheet名和dtype修复)
# ==============================================================================
print("--- 步骤 1: 正在加载数据 ---")
try:
    # --- 关键修复 1: 明确指定sheet_name ---
    sheet_name_results = '全部详细结果'
    print(f"  -> 正在从 Sheet '{sheet_name_results}' 加载回归结果...")
    
    # --- 关键修复 2: 强制指定数值列的数据类型 ---
    xls_results = pd.ExcelFile(results_file_path)
    # 检查sheet是否存在
    if sheet_name_results not in xls_results.sheet_names:
        raise ValueError(f"在文件 {results_file_path} 中找不到名为 '{sheet_name_results}' 的Sheet。")
    
    df_temp = pd.read_excel(xls_results, sheet_name=sheet_name_results, nrows=0)
    dtype_dict = {col: float for col in df_temp.columns if 'β' in col or 'p_' in col or 'R-squared' in col or 'k_' in col}
    
    df_results = pd.read_excel(results_file_path, sheet_name=sheet_name_results, index_col=0, dtype=dtype_dict)
    
    df_results.index = df_results.index.astype(str).str.strip()
    df_results.index.name = '行业分类'
    print("  -> 回归分析结果加载成功。")

    # (业扩数据加载部分保持不变)
    all_cap_data_long = []
    xls_cap = pd.ExcelFile(raw_cap_data_path)
    available_sheets = xls_cap.sheet_names
    for var_name, sheet_name in cap_sheet_map.items():
        if sheet_name in available_sheets:
            df = pd.read_excel(raw_cap_data_path, sheet_name=sheet_name)
            df['分类'] = df['分类'].astype(str).str.strip()
            valid_date_cols = [col for col in df.columns if re.match(r'^\d{6}$', str(col).split('.')[0])]
            df = df[['分类'] + valid_date_cols]
            df_long = df.melt(id_vars='分类', var_name='date', value_name=var_name)
            df_long['date'] = pd.to_datetime(df_long['date'].astype(str), format='%Y%m')
            df_long[var_name] = pd.to_numeric(df_long[var_name], errors='coerce')
            all_cap_data_long.append(df_long.set_index(['分类', 'date']))
    if not all_cap_data_long: raise ValueError("未能加载任何业扩数据。")
    df_cap_raw = pd.concat(all_cap_data_long, axis=1)
    print("  -> 原始业扩数据加载成功。")
except Exception as e:
    raise RuntimeError(f"加载数据时出错: {e}")

# ==============================================================================
# 2. 计算业扩贡献电量 (无p值过滤)
# ==============================================================================
print("\n--- 步骤 2: 正在计算业扩贡献电量 (无p值过滤) ---")
df_cap_diff = df_cap_raw.groupby(level='分类').diff(1)
all_predictions = []
for industry_name, row in df_results.iterrows():
    print(f"  -> 正在为行业 '{industry_name}' 进行计算...")
    if industry_name not in df_cap_diff.index.get_level_values('分类'):
        print(f"     - 警告: 在业扩数据中未找到行业 '{industry_name}'，跳过。")
        continue
    industry_cap_diff = df_cap_diff.loc[industry_name]
    for month_str in prediction_months:
        target_month = pd.to_datetime(month_str, format='%Y%m')
        total_contribution_current_year, total_contribution_last_year = 0, 0
        target_month_last_year = target_month - pd.DateOffset(years=1)
        for cap_var in cap_sheet_map.keys():
            k = int(row.get(f'k_{cap_var}', 0))
            beta = row.get(f'β_{cap_var}', 0)
            required_date_current = target_month - pd.DateOffset(months=k)
            if required_date_current in industry_cap_diff.index:
                cap_increment = industry_cap_diff.loc[required_date_current, cap_var]
                if pd.notna(cap_increment): total_contribution_current_year += beta * cap_increment
            required_date_last = target_month_last_year - pd.DateOffset(months=k)
            if required_date_last in industry_cap_diff.index:
                cap_increment_last = industry_cap_diff.loc[required_date_last, cap_var]
                if pd.notna(cap_increment_last): total_contribution_last_year += beta * cap_increment_last
        all_predictions.append({'行业分类': industry_name, '预测月份': target_month, '业扩贡献电量': total_contribution_current_year, '去年同期贡献': total_contribution_last_year})
df_predictions = pd.DataFrame(all_predictions)

# ==============================================================================
# 3. 结果展示
# ==============================================================================
print("\n--- 步骤 3: 结果展示 ---")
if df_predictions.empty:
    print("未能计算出任何预测结果。")
else:
    df_pivot = df_predictions.pivot(index='行业分类', columns='预测月份', values=['业扩贡献电量', '去年同期贡献'])
    cols_ordered, new_cols = [], []
    for month_dt in sorted(df_predictions['预测月份'].unique()):
        cols_ordered.extend([('业扩贡献电量', month_dt), ('去年同期贡献', month_dt)])
        month_str_display = month_dt.strftime('%Y-%m')
        new_cols.extend([f'{month_str_display} 贡献 (万kWh)', f'{month_dt - pd.DateOffset(years=1):%Y-%m} 同期贡献 (万kWh)'])
    df_pivot = df_pivot[cols_ordered]; df_pivot.columns = new_cols
    print("\n【情景模拟】业扩潜在贡献电量预测 (单位: 万kWh):")
    display(df_pivot.style.format("{:,.6f}").background_gradient(cmap='RdYlGn', axis=None, low=0.4, high=0.4))
    output_prediction_path = r'E:\A智网\业扩分析\8月分析\7月业扩月度报告\8.15\7月业扩月度报告贡献电量预测.xlsx'
    df_pivot.to_excel(output_prediction_path)
    print(f"\n预测结果已保存到: {output_prediction_path}")

--- 步骤 1: 正在加载数据 ---
  -> 正在从 Sheet '全部详细结果' 加载回归结果...
  -> 回归分析结果加载成功。
  -> 原始业扩数据加载成功。

--- 步骤 2: 正在计算业扩贡献电量 (无p值过滤) ---
  -> 正在为行业 '全行业总计' 进行计算...
  -> 正在为行业 'A、全行业用电合计' 进行计算...
  -> 正在为行业 '第一产业' 进行计算...
  -> 正在为行业 '第二产业' 进行计算...
  -> 正在为行业 '第三产业' 进行计算...
  -> 正在为行业 'B、城乡居民生活用电合计' 进行计算...
  -> 正在为行业 '城镇居民' 进行计算...
  -> 正在为行业 '乡村居民' 进行计算...
  -> 正在为行业 '全行业用电分类' 进行计算...
  -> 正在为行业 '一、农、林、牧、渔业' 进行计算...
  -> 正在为行业 '1.农业' 进行计算...
  -> 正在为行业 '2.林业' 进行计算...
  -> 正在为行业 '3.畜牧业' 进行计算...
  -> 正在为行业 '4.渔业' 进行计算...
  -> 正在为行业 '5.农、林、牧、渔专业及辅助性活动' 进行计算...
  -> 正在为行业 '其中：排灌' 进行计算...
  -> 正在为行业 '二、工业' 进行计算...
  -> 正在为行业 '（一）采矿业' 进行计算...
  -> 正在为行业 '1.煤炭开采和洗选业' 进行计算...
  -> 正在为行业 '2.石油和天燃气开采业' 进行计算...
  -> 正在为行业 '3.黑色金属矿采选业' 进行计算...
  -> 正在为行业 '4.有色金属矿采选业' 进行计算...
  -> 正在为行业 '5.非金属矿采选业' 进行计算...
  -> 正在为行业 '6.其他采矿业' 进行计算...
  -> 正在为行业 '（二）制造业' 进行计算...
  -> 正在为行业 '1.农副食品加工业' 进行计算...
  -> 正在为行业 '2.食品制造业' 进行计算...
  -> 正在为行业 '3.酒、饮料及精制茶制造业' 进行计算...
  -> 正在为行业 '4.烟草制品业' 进行计算...
  -> 正在为行业 '5.纺织业' 进行计算...
 

,2025-07 贡献 (万kWh),2024-07 同期贡献 (万kWh),2025-08 贡献 (万kWh),2024-08 同期贡献 (万kWh),2025-09 贡献 (万kWh),2024-09 同期贡献 (万kWh)
行业分类,,,,,,
1.农业,-358.569377,"-1,047.912161",226.591806,-286.476556,135.145964,277.722851
1.农副食品加工业,494.768943,-301.793622,158.575408,"1,069.775270",64.945138,363.339093
1.房屋建筑业,499.199931,-999.512740,-52.544703,625.401762,-598.079409,582.121313
1.煤炭开采和洗选业,-38.186177,0.000000,14.467980,3.009322,0.000000,-3.009322
1.电信、广播电视和卫星传输服务,18.789478,-74.776843,-6.919893,-50.711845,-66.627469,-31.815659
1.电力、热力生产和供应业,"-52,851.431937","14,584.698664","30,090.248299","-6,566.535456","1,909.098100","-1,602.612038"
1.科学研究和技术服务业,288.458292,129.276732,-642.858091,271.628906,326.106484,-93.697083
1.铁路运输业,624.649826,101.779097,-464.435109,-39.725049,-76.203044,-124.108096
10.造纸和纸制品业,"1,330.991418","8,006.197711","-1,178.713829","-8,035.961487",-118.764206,-132.019102



预测结果已保存到: E:\A智网\业扩分析\8月分析\7月业扩月度报告\8.15\7月业扩月度报告贡献电量预测.xlsx
